### 6-2．데이터의 준비
#### 데이터의 리드

In [1]:
import pandas as pd

# 파일 리드
dat_df = pd.read_csv('energydata.csv', sep=',')

# 선두에서 5행까지 표시
dat_df.head()

date  Appliances  lights     T1       RH_1    T2       RH_2  \
0  2016-01-11 17:00:00          60      30  19.89  47.596667  19.2  44.790000   
1  2016-01-11 17:10:00          60      30  19.89  46.693333  19.2  44.722500   
2  2016-01-11 17:20:00          50      30  19.89  46.300000  19.2  44.626667   
3  2016-01-11 17:30:00          50      40  19.89  46.066667  19.2  44.590000   
4  2016-01-11 17:40:00          60      40  19.89  46.333333  19.2  44.530000   

      T3       RH_3         T4       RH_4         T5   RH_5  
0  19.79  44.730000  19.000000  45.566667  17.166667  55.20  
1  19.79  44.790000  19.000000  45.992500  17.166667  55.20  
2  19.79  44.933333  18.926667  45.890000  17.166667  55.09  
3  19.79  45.000000  18.890000  45.723333  17.166667  55.09  
4  19.79  45.000000  18.890000  45.530000  17.200000  55.09

#### 결손값의 보완

In [2]:
# 결손값이 포함된 행을 확인
dat_df[8585:8588]

date  Appliances  lights    T1       RH_1    T2  \
8585  2016-03-11 07:50:00          70       0  20.2  37.160000  16.5   
8586  2016-03-11 08:00:00         280       0  20.2  37.463333  16.6   
8587  2016-03-11 08:10:00         180      10  20.2  37.530000  16.6   

           RH_2     T3       RH_3         T4       RH_4         T5       RH_5  
8585  42.430000  20.39  37.500000  19.700000  34.360000  18.000000  46.660000  
8586  42.766667    NaN  37.500000  19.700000  34.633333  18.066667  46.193333  
8587  42.966667  20.29  37.363333  19.666667  34.723333  18.100000  45.723333

In [3]:
# 하나 전행의 값으로 보완
dat_df[8585:8588].fillna(method='ffill')

date  Appliances  lights    T1       RH_1    T2  \
8585  2016-03-11 07:50:00          70       0  20.2  37.160000  16.5   
8586  2016-03-11 08:00:00         280       0  20.2  37.463333  16.6   
8587  2016-03-11 08:10:00         180      10  20.2  37.530000  16.6   

           RH_2     T3       RH_3         T4       RH_4         T5       RH_5  
8585  42.430000  20.39  37.500000  19.700000  34.360000  18.000000  46.660000  
8586  42.766667  20.39  37.500000  19.700000  34.633333  18.066667  46.193333  
8587  42.966667  20.29  37.363333  19.666667  34.723333  18.100000  45.723333

In [4]:
# 결손값의 선형보완
dat_df = dat_df.interpolate()

# 결손값이 포함된 행을 재확인
dat_df[8585:8588]

date  Appliances  lights    T1       RH_1    T2  \
8585  2016-03-11 07:50:00          70       0  20.2  37.160000  16.5   
8586  2016-03-11 08:00:00         280       0  20.2  37.463333  16.6   
8587  2016-03-11 08:10:00         180      10  20.2  37.530000  16.6   

           RH_2     T3       RH_3         T4       RH_4         T5       RH_5  
8585  42.430000  20.39  37.500000  19.700000  34.360000  18.000000  46.660000  
8586  42.766667  20.34  37.500000  19.700000  34.633333  18.066667  46.193333  
8587  42.966667  20.29  37.363333  19.666667  34.723333  18.100000  45.723333

#### 시간축의 작성

In [5]:
# date를 object형에서 datetime형으로 변환
dat_df['date'] = pd.to_datetime(dat_df['date'], format='%Y-%m-%d %H:%M:%S')

# 전행과의 시간차를 계산
dat_df['dif_min'] = dat_df['date'].diff().dt.total_seconds()/60
# 선두 요소의 결손을 0으로 메꾼다
dat_df['dif_min'] = dat_df['dif_min'].fillna(0)

# 경과시간(분)을 계산
dat_df['cum_min'] = dat_df['dif_min'].cumsum()

# 선두에서 5행까지 표시
dat_df[['date', 'cum_min']].head()

date  cum_min
0 2016-01-11 17:00:00      0.0
1 2016-01-11 17:10:00     10.0
2 2016-01-11 17:20:00     20.0
3 2016-01-11 17:30:00     30.0
4 2016-01-11 17:40:00     40.0

In [6]:
# 연습문제4
# 경과시간(6시간)을 계산
dat_df['cum_6hour'] = (dat_df['cum_min']/360).round(2).astype(int)

# 시간ID의 확인
print(dat_df['cum_6hour'].unique())
# 선두에서 50행까지 표시
dat_df[['date', 'cum_min', 'cum_6hour']].head(50)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

date  cum_min  cum_6hour
0  2016-01-11 17:00:00      0.0          0
1  2016-01-11 17:10:00     10.0          0
2  2016-01-11 17:20:00     20.0          0
3  2016-01-11 17:30:00     30.0          0
4  2016-01-11 17:40:00     40.0          0
5  2016-01-11 17:50:00     50.0          0
6  2016-01-11 18:00:00     60.0          0
7  2016-01-11 18:10:00     70.0          0
8  2016-01-11 18:20:00     80.0          0
9  2016-01-11 18:30:00     90.0          0
10 2016-01-11 18:40:00    100.0          0
11 2016-01-11 18:50:00    110.0          0
12 2016-01-11 19:00:00    120.0          0
13 2016-01-11 19:10:00    130.0          0
14 2016-01-11 19:20:00    140.0          0
15 2016-01-11 19:30:00    150.0          0
16 2016-01-11 19:40:00    160.0          0
17 2016-01-11 19:50:00    170.0          0
18 2016-01-11 20:00:00    180.0          0
19 2016-01-11 20:10:00    190.0          0
20 2016-01-11 20:20:00    200.0          0
21 2016-01-11 20:30:00    210.0          0
22 2016-01-11 20:40:00    220.0          0
23 2016-01-11 20:50:00    230.0          0
24 2016-01-11 21:00:00    240.0          0
25 2016-01-11 21:10:00    250.0          0
26 2016-01-11 21:20:00    260.0          0
27 2016-01-11 21:30:00    270.0          0
28 2016-01-11 21:40:00    280.0          0
29 2016-01-11 21:50:00    290.0          0
30 2016-01-11 22:00:00    300.0          0
31 2016-01-11 22:10:00    310.0          0
32 2016-01-11 22:20:00    320.0          0
33 2016-01-11 22:30:00    330.0          0
34 2016-01-11 22:40:00    340.0          0
35 2016-01-11 22:50:00    350.0          0
36 2016-01-11 23:00:00    360.0          1
37 2016-01-11 23:10:00    370.0          1
38 2016-01-11 23:20:00    380.0          1
39 2016-01-11 23:30:00    390.0          1
40 2016-01-11 23:40:00    400.0          1
41 2016-01-11 23:50:00    410.0          1
42 2016-01-12 00:00:00    420.0          1
43 2016-01-12 00:10:00    430.0          1
44 2016-01-12 00:20:00    440.0          1
45 2016-01-12 00:30:00    450.0          1
46 2016-01-12 00:40:00    460.0          1
47 2016-01-12 00:50:00    470.0          1
48 2016-01-12 01:00:00    480.0          1
49 2016-01-12 01:10:00    490.0          1

#### 특징량의 작성 1

In [7]:
# 6시간마다 평균값을 계산
dat_df = dat_df.drop(['date', 'dif_min', 'cum_min'], axis=1)
dat_df_mean = dat_df.groupby('cum_6hour').mean()

# 데이터 세트의 사이즈를 확인
print(dat_df_mean.shape)
# 선두에서 5행까지 표시
dat_df_mean.head()

(484, 12)


Appliances     lights         T1       RH_1         T2       RH_2  \
cum_6hour                                                                      
0          150.000000  31.944444  20.688056  47.345394  20.023148  44.973287   
1           81.944444   7.222222  20.858287  45.627708  20.227315  44.286435   
2           83.333333   5.555556  19.860463  47.019213  19.140608  44.876698   
3          101.111111   0.000000  20.037500  44.843009  19.103377  43.495476   
4           78.611111   7.222222  19.947963  43.275394  19.113860  42.606633   

                  T3       RH_3         T4       RH_4         T5       RH_5  
cum_6hour                                                                    
0          20.111481  46.041875  19.255972  47.379074  17.761597  60.359861  
1          20.213588  45.530833  20.740093  46.503634  18.898519  50.261273  
2          20.149722  45.269306  19.349259  45.893611  18.163125  50.817454  
3          19.907778  44.802407  19.748287  43.536204  17.835370  50.444111  
4          19.788704  44.312037  19.555093  42.804491  17.477870  50.732546

In [8]:
# 연습문제5
# 6시간마다 표준편차를 계산
dat_df_std = dat_df.groupby('cum_6hour').std()

# 데이터 세트의 사이즈를 확인
print(dat_df_std.shape)
# 선두에서 5행까지 표시
dat_df_std.head()

(484, 12)


Appliances     lights        T1      RH_1        T2      RH_2  \
cum_6hour                                                                  
0          131.366445  13.901582  0.672443  2.998070  0.653222  0.848637   
1          101.497908   9.137399  0.460572  0.811346  0.476669  0.342190   
2          104.853639   9.085135  0.103157  1.039938  0.167655  0.318816   
3           95.999339   0.000000  0.059026  1.091231  0.069944  0.617057   
4           83.773599  12.097488  0.151195  0.561101  0.187641  0.418215   

                 T3      RH_3        T4      RH_4        T5      RH_5  
cum_6hour                                                              
0          0.278809  1.185530  0.485747  1.282966  0.791008  9.891611  
1          0.082826  0.103003  0.560148  0.511257  0.272610  0.339389  
2          0.142110  0.261039  0.273491  0.776746  0.178471  1.138170  
3          0.046289  0.068936  0.274278  0.622986  0.164863  0.792715  
4          0.048003  0.180349  0.341428  0.430826  0.075740  0.247231

In [9]:
# 평균값과 표준편차 데이터 프레임을 인덱스로 결합
dat_features = pd.merge(dat_df_mean, dat_df_std, left_index=True, right_index=True)

# 
print(dat_features.shape)
# 선두에서 5행까지 표시
dat_features.head()

(484, 24)


Appliances_x   lights_x       T1_x     RH_1_x       T2_x  \
cum_6hour                                                             
0            150.000000  31.944444  20.688056  47.345394  20.023148   
1             81.944444   7.222222  20.858287  45.627708  20.227315   
2             83.333333   5.555556  19.860463  47.019213  19.140608   
3            101.111111   0.000000  20.037500  44.843009  19.103377   
4             78.611111   7.222222  19.947963  43.275394  19.113860   

              RH_2_x       T3_x     RH_3_x       T4_x     RH_4_x  ...  \
cum_6hour                                                         ...   
0          44.973287  20.111481  46.041875  19.255972  47.379074  ...   
1          44.286435  20.213588  45.530833  20.740093  46.503634  ...   
2          44.876698  20.149722  45.269306  19.349259  45.893611  ...   
3          43.495476  19.907778  44.802407  19.748287  43.536204  ...   
4          42.606633  19.788704  44.312037  19.555093  42.804491  ...   

               T1_y    RH_1_y      T2_y    RH_2_y      T3_y    RH_3_y  \
cum_6hour                                                               
0          0.672443  2.998070  0.653222  0.848637  0.278809  1.185530   
1          0.460572  0.811346  0.476669  0.342190  0.082826  0.103003   
2          0.103157  1.039938  0.167655  0.318816  0.142110  0.261039   
3          0.059026  1.091231  0.069944  0.617057  0.046289  0.068936   
4          0.151195  0.561101  0.187641  0.418215  0.048003  0.180349   

               T4_y    RH_4_y      T5_y    RH_5_y  
cum_6hour                                          
0          0.485747  1.282966  0.791008  9.891611  
1          0.560148  0.511257  0.272610  0.339389  
2          0.273491  0.776746  0.178471  1.138170  
3          0.274278  0.622986  0.164863  0.792715  
4          0.341428  0.430826  0.075740  0.247231  

[5 rows x 24 columns]

#### 목적변수의 작성

In [10]:
# 파일 리드
event_df = pd.read_csv('event.csv', sep=',')

# 선두에서 5행까지 표시
event_df.head()

date
0  2016-01-12 10:00:00
1  2016-01-13 21:00:00
2  2016-01-14 10:00:00
3  2016-01-14 16:00:00
4  2016-01-14 17:00:00

In [11]:
import datetime as dt

# date를 object형에서 datetime형으로 변환
event_df['date'] = pd.to_datetime(event_df['date'], format='%Y-%m-%d %H:%M:%S')

# 기준일시를 설명변수의 세트에 포함시킨다
base_time = '2016-01-11 17:00:00'
# 표준일시와 각 일시의 차이를 계산
event_df['dif_min'] = event_df['date'] - dt.datetime.strptime(base_time, '%Y-%m-%d %H:%M:%S')
event_df['dif_min'] = event_df['dif_min'].dt.total_seconds()/60
# 6시간 단위의 시각ID를 계산
event_df['cum_6hour'] = (event_df['dif_min']/360).round(2).astype(int)

# 선두에서 5행까지 표시
event_df.head()

date  dif_min  cum_6hour
0 2016-01-12 10:00:00   1020.0          2
1 2016-01-13 21:00:00   3120.0          8
2 2016-01-14 10:00:00   3900.0         10
3 2016-01-14 16:00:00   4260.0         11
4 2016-01-14 17:00:00   4320.0         12

In [12]:
# 이상이 발생한것으로 가정하여 플래그에 1을 입력
event_df['event'] = 1

# cum_6hour와 event만 추출
event_df = event_df[['cum_6hour', 'event']]
event_df = event_df[~event_df.duplicated()]

# cum_6hour를 인덱스로 변경
event_df = event_df.set_index(['cum_6hour'])

# 선두에서 5행까지 표시
event_df.head()

event
cum_6hour       
2              1
8              1
10             1
11             1
12             1

In [13]:
# 특징량과 목적변수를 인덱스로 결합
dat_event = dat_features.join(event_df, how='left')
dat_event = dat_event.fillna(0)

# 데이터 세트의 사이즈를 확인
print(dat_event.shape)
# 선두에서 5행까지 표시
dat_event.head()

(484, 25)


Appliances_x   lights_x       T1_x     RH_1_x       T2_x  \
cum_6hour                                                             
0            150.000000  31.944444  20.688056  47.345394  20.023148   
1             81.944444   7.222222  20.858287  45.627708  20.227315   
2             83.333333   5.555556  19.860463  47.019213  19.140608   
3            101.111111   0.000000  20.037500  44.843009  19.103377   
4             78.611111   7.222222  19.947963  43.275394  19.113860   

              RH_2_x       T3_x     RH_3_x       T4_x     RH_4_x  ...  \
cum_6hour                                                         ...   
0          44.973287  20.111481  46.041875  19.255972  47.379074  ...   
1          44.286435  20.213588  45.530833  20.740093  46.503634  ...   
2          44.876698  20.149722  45.269306  19.349259  45.893611  ...   
3          43.495476  19.907778  44.802407  19.748287  43.536204  ...   
4          42.606633  19.788704  44.312037  19.555093  42.804491  ...   

             RH_1_y      T2_y    RH_2_y      T3_y    RH_3_y      T4_y  \
cum_6hour                                                               
0          2.998070  0.653222  0.848637  0.278809  1.185530  0.485747   
1          0.811346  0.476669  0.342190  0.082826  0.103003  0.560148   
2          1.039938  0.167655  0.318816  0.142110  0.261039  0.273491   
3          1.091231  0.069944  0.617057  0.046289  0.068936  0.274278   
4          0.561101  0.187641  0.418215  0.048003  0.180349  0.341428   

             RH_4_y      T5_y    RH_5_y  event  
cum_6hour                                       
0          1.282966  0.791008  9.891611    0.0  
1          0.511257  0.272610  0.339389    0.0  
2          0.776746  0.178471  1.138170    1.0  
3          0.622986  0.164863  0.792715    0.0  
4          0.430826  0.075740  0.247231    0.0  

[5 rows x 25 columns]

#### 특징량의 작성 2

In [14]:
# 처음 6시간을 데이터 세트로 지정
tmp = dat_df[dat_df['cum_6hour'] == 0]
# cum_6hour항목을 제외
tmp = tmp.drop(['cum_6hour'], axis=1)

# 슬라이드창의 평균값을 계산
tmp = tmp.rolling(6).mean()
tmp = tmp.dropna()

# 창별로 특징량을 확인
tmp

Appliances     lights         T1       RH_1         T2       RH_2  \
5    55.000000  35.000000  19.890000  46.502778  19.200000  44.626528   
6    55.000000  38.333333  19.890000  46.197778  19.200000  44.578194   
7    55.000000  41.666667  19.884444  46.008889  19.200000  44.541111   
8    56.666667  43.333333  19.867778  45.891806  19.200000  44.508889   
9    60.000000  43.333333  19.862222  45.895694  19.205000  44.477222   
10   88.333333  48.333333  19.868333  45.817361  19.231111  44.455556   
11  176.666667  51.666667  19.897778  45.879028  19.268889  44.438889   
12  238.333333  51.666667  19.938333  46.251250  19.330000  44.422222   
13  270.000000  50.000000  20.005556  47.445694  19.418333  44.522222   
14  276.666667  45.000000  20.111667  49.161667  19.527778  44.755556   
15  281.666667  40.000000  20.230000  50.462222  19.661667  45.148333   
16  258.333333  30.000000  20.363889  51.595000  19.796667  45.618889   
17  173.333333  25.000000  20.495556  52.805556  19.925556  46.061667   
18  115.000000  21.666667  20.621667  53.337778  20.031111  46.383333   
19   96.666667  21.666667  20.726667  52.850000  20.109444  46.537778   
20  100.000000  23.333333  20.803889  51.606667  20.166667  46.519306   
21  115.000000  28.333333  20.870000  50.563333  20.204444  46.314861   
22  118.333333  33.333333  20.920556  49.630556  20.225000  45.981528   
23  125.000000  35.000000  20.961111  48.453333  20.251111  45.632639   
24  130.000000  35.000000  21.001667  47.597778  20.288889  45.370972   
25  125.000000  31.666667  21.053333  46.931111  20.338889  45.159861   
26  121.666667  33.333333  21.120000  46.508889  20.394444  45.054444   
27  106.666667  30.000000  21.185556  46.207778  20.462222  44.993333   
28  105.000000  26.666667  21.245000  45.978889  20.530556  44.966111   
29  103.333333  23.333333  21.311667  45.768333  20.587778  44.961111   
30  103.333333  21.666667  21.372778  45.612778  20.648333  44.940000   
31  151.666667  21.666667  21.428333  45.473889  20.696667  44.873333   
32  201.666667  20.000000  21.480000  45.285000  20.756111  44.728889   
33  250.000000  21.666667  21.520556  45.073889  20.805556  44.528889   
34  273.333333  21.666667  21.555556  44.852222  20.855556  44.324444   
35  266.666667  21.666667  21.572222  44.663333  20.905556  44.118889   

           T3       RH_3         T4       RH_4         T5       RH_5  
5   19.790000  44.897778  18.932778  45.738750  17.166667  55.116667  
6   19.790000  44.926111  18.914444  45.775972  17.155556  55.077778  
7   19.780000  44.944444  18.896111  45.754444  17.144444  55.027778  
8   19.770000  44.920556  18.890000  45.737778  17.144444  55.012778  
9   19.770000  44.897778  18.890000  45.800000  17.133333  54.997778  
10  19.770000  44.881111  18.890000  45.950000  17.116667  54.982778  
11  19.770000  44.863333  18.908333  46.066667  17.111111  54.977778  
12  19.786667  44.863333  18.926667  46.162222  17.111111  54.998333  
13  19.813333  44.962222  18.945000  46.388889  17.111111  55.042222  
14  19.863889  45.418333  18.963333  46.873333  17.100000  55.125556  
15  19.915556  46.018889  18.981667  47.272222  17.108333  55.299306  
16  19.983889  46.623889  18.987778  47.560000  17.119444  55.547639  
17  20.052222  47.227361  18.969444  47.815556  17.136111  55.869861  
18  20.103889  47.776250  18.963333  48.193333  17.152778  56.198194  
19  20.150000  48.132917  18.968889  48.572778  17.235000  58.792083  
20  20.172222  48.095139  18.998056  48.781250  17.396111  62.708750  
21  20.188889  47.855139  19.041389  48.982361  17.482222  66.656667  
22  20.198889  47.571250  19.107500  49.150694  17.554444  70.451111  
23  20.213889  47.268889  19.190833  49.227917  17.615556  74.027778  
24  20.228889  46.975556  19.261944  49.155139  17.708889  78.261111  
25  20.266111  46.740556  19.321389  48.871806  17.836111  80.238333  
26  20.303333  46.538889  19.357222  48.473889  17.884444  78.903333  
27  20.335000  46.382778  19.37888

In [15]:
import numpy as np

# 특징량을 결합
tmp2 = (np.array(tmp['Appliances']).tolist() + np.array(tmp['lights']).tolist() + 
        np.array(tmp['T1']).tolist() + np.array(tmp['RH_1']).tolist() + 
        np.array(tmp['T2']).tolist() + np.array(tmp['RH_2']).tolist() + 
        np.array(tmp['T3']).tolist() + np.array(tmp['RH_3']).tolist() + 
        np.array(tmp['T4']).tolist() + np.array(tmp['RH_4']).tolist() + 
        np.array(tmp['T5']).tolist() + np.array(tmp['RH_5']).tolist())

# 특징량의 길이를 확인
print(len(tmp2))
# 특징량을 행방향으로 정렬시켜서 확인
pd.DataFrame(tmp2).T

372


0     1     2          3     4          5           6           7    \
0  55.0  55.0  55.0  56.666667  60.0  88.333333  176.666667  238.333333   

     8           9    ...        362    363        364        365        366  \
0  270.0  276.666667  ...  78.903333  76.02  72.648889  69.037778  64.516667   

         367    368        369        370        371  
0  59.767222  56.83  55.170417  53.997083  53.129306  

[1 rows x 372 columns]

In [16]:
# 연습문제6
# 6시간 단위의 시간ID를 취득
hid = dat_df['cum_6hour'].unique()

# 슬라이드창의 특징량을 보존
dat_slide_features = []

for i in range(len(hid)):
    # 6시간 단위의 데이터 세트를 취득
    tmp = dat_df[dat_df['cum_6hour'] == i]
    # cum_6hour 항목을 제외
    tmp = tmp.drop(['cum_6hour'], axis=1)

    # 슬라이드창의 평균값을 계산
    tmp_mean = tmp.rolling(6).mean()
    tmp_mean = tmp_mean.dropna()
    # 슬라이드창의 표준편차를 계산
    tmp_std = tmp.rolling(6).mean()
    tmp_std = tmp_std.dropna()    

    # 특징량을 결합
    tmp2 = (np.array(tmp_mean['Appliances']).tolist() + np.array(tmp_mean['lights']).tolist() + 
            np.array(tmp_mean['T1']).tolist() + np.array(tmp_mean['RH_1']).tolist() + 
            np.array(tmp_mean['T2']).tolist() + np.array(tmp_mean['RH_2']).tolist() + 
            np.array(tmp_mean['T3']).tolist() + np.array(tmp_mean['RH_3']).tolist() + 
            np.array(tmp_mean['T4']).tolist() + np.array(tmp_mean['RH_4']).tolist() + 
            np.array(tmp_mean['T5']).tolist() + np.array(tmp_mean['RH_5']).tolist() + 
            np.array(tmp_std['Appliances']).tolist() + np.array(tmp_std['lights']).tolist() + 
            np.array(tmp_std['T1']).tolist() + np.array(tmp_std['RH_1']).tolist() + 
            np.array(tmp_std['T2']).tolist() + np.array(tmp_std['RH_2']).tolist() + 
            np.array(tmp_std['T3']).tolist() + np.array(tmp_std['RH_3']).tolist() + 
            np.array(tmp_std['T4']).tolist() + np.array(tmp_std['RH_4']).tolist() + 
            np.array(tmp_std['T5']).tolist() + np.array(tmp_std['RH_5']).tolist())

    dat_slide_features.append(tmp2)

In [17]:
# 특징량을 데이터 프레임으로 변환
dat_slide_features = pd.DataFrame(dat_slide_features)

# 데이터 세트의 사이즈를 확인
print(dat_slide_features.shape)
# 데이터 세트의 선두에서 5행까지 확인
dat_slide_features.head()

(484, 744)


0           1           2           3           4           5    \
0   55.000000   55.000000   55.000000   56.666667   60.000000   88.333333   
1   56.666667   53.333333   48.333333   46.666667   43.333333   85.000000   
2   43.333333   43.333333   46.666667   46.666667   45.000000   41.666667   
3   55.000000   56.666667   80.000000  108.333333  126.666667  181.666667   
4  218.333333  221.666667  226.666667  203.333333  146.666667   91.666667   

          6           7           8           9    ...        734        735  \
0  176.666667  238.333333  270.000000  276.666667  ...  78.903333  76.020000   
1  141.666667  198.333333  255.000000  268.333333  ...  50.090000  50.080000   
2   40.000000   38.333333   35.000000   38.333333  ...  52.633333  52.572222   
3  186.666667  186.666667  203.333333  188.333333  ...  51.000000  51.000000   
4   58.333333   53.333333   46.666667   45.000000  ...  50.601111  50.536111   

         736        737        738        739        740        741  \
0  72.648889  69.037778  64.516667  59.767222  56.830000  55.170417   
1  50.065000  50.050000  50.035000  50.020000  50.005000  50.000000   
2  52.305556  51.950000  51.473333  50.951111  50.407778  49.995556   
3  51.000000  51.000000  51.000000  51.010000  51.010000  51.010000   
4  50.481111  50.437222  50.397222  50.362222  50.338889  50.350000   

         742        743  
0  53.997083  53.129306  
1  50.000000  50.000000  
2  49.783333  49.640000  
3  51.004444  50.987778  
4  50.371667  50.427778  

[5 rows x 744 columns]

In [18]:
# 특징량과 목적변수를 결합
tmp3 = dat_event[['event']]
dat_event2 = pd.concat([dat_slide_features, tmp3], axis=1)

# 데이터 세트의 사이즈를 확인
print(dat_event2.shape)
# 데이터 세트의 선두에서 5행까지 확인
dat_event2.head()

(484, 745)


0           1           2           3           4           5  \
0   55.000000   55.000000   55.000000   56.666667   60.000000   88.333333   
1   56.666667   53.333333   48.333333   46.666667   43.333333   85.000000   
2   43.333333   43.333333   46.666667   46.666667   45.000000   41.666667   
3   55.000000   56.666667   80.000000  108.333333  126.666667  181.666667   
4  218.333333  221.666667  226.666667  203.333333  146.666667   91.666667   

            6           7           8           9  ...        735        736  \
0  176.666667  238.333333  270.000000  276.666667  ...  76.020000  72.648889   
1  141.666667  198.333333  255.000000  268.333333  ...  50.080000  50.065000   
2   40.000000   38.333333   35.000000   38.333333  ...  52.572222  52.305556   
3  186.666667  186.666667  203.333333  188.333333  ...  51.000000  51.000000   
4   58.333333   53.333333   46.666667   45.000000  ...  50.536111  50.481111   

         737        738        739        740        741        742  \
0  69.037778  64.516667  59.767222  56.830000  55.170417  53.997083   
1  50.050000  50.035000  50.020000  50.005000  50.000000  50.000000   
2  51.950000  51.473333  50.951111  50.407778  49.995556  49.783333   
3  51.000000  51.000000  51.010000  51.010000  51.010000  51.004444   
4  50.437222  50.397222  50.362222  50.338889  50.350000  50.371667   

         743  event  
0  53.129306    0.0  
1  50.000000    0.0  
2  49.640000    1.0  
3  50.987778    0.0  
4  50.427778    0.0  

[5 rows x 745 columns]